In [15]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import pickle
import os
df1 = pd.read_csv('QM9_129440_MLtraining.csv')
df2 = pd.read_csv('QM9_49762_MLtraining.csv')
df2 = df2.dropna()

# Remove infinite values
df2 = df2.replace([np.inf, -np.inf], np.nan).dropna()

to_drop = ['canonical_smiles']  # Thêm các features khác nếu cần
df2 = df2.drop(columns=to_drop)

missing_threshold = 0.4
missing_percent = df2.isnull().mean()
to_drop = missing_percent[missing_percent > missing_threshold].index
df2 = df2.drop(columns=to_drop)

# Loại bỏ features có variance thấp
from sklearn.feature_selection import VarianceThreshold
selector = VarianceThreshold(threshold=0.08)  # Điều chỉnh threshold
selector.fit(df2.select_dtypes(include=['float64', 'int64']))
low_variance_cols = df2.columns[~selector.get_support()]
df2 = df2.drop(columns=low_variance_cols)

In [ ]:
def save_model(self, model_name, model_type, method):
        pkl_filename= f'{model_type}_{type(model_name).__name__}_{method}.pkl'
        with open(pkl_filename, 'wb') as file:  
            pickle.dump(model_name, file)
        with open(pkl_filename, 'rb') as file:  
            saved_model = pickle.load(file)
                
        print(saved_model)
# Tính toán MAPE
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# Giả sử df2 là dataframe chứa dữ liệu của bạn
features = [col for col in df2.columns if col not in 
            ['canonical_smiles', 'mps_pred', 'bps_pred', 'fps_pred']]

targets = ['bps_pred', 'mps_pred', 'fps_pred']
methods = ['BPS', 'MPS', 'FPS']
cv = 5

# Hàm để lấy feature importance
def get_feature_importance(model, feature_names):
    if hasattr(model.named_steps['model'], 'feature_importances_'):
        importances = model.named_steps['model'].feature_importances_
    elif hasattr(model.named_steps['model'], 'coef_'):
        importances = model.named_steps['model'].coef_
    else:
        return None
    
    feature_importance = pd.DataFrame({'feature': feature_names, 
                                     'importance': importances})
    return feature_importance.sort_values('importance', ascending=False)

# Xây dựng mô hình cho từng target
for i, target in enumerate(targets):
    print(f"\n=== Building models for {target} ===")
    print("="*50)
    
    X = df2[features]
    # Handle infinity values by replacing with mean
    inf_mask = X.isin([np.inf, -np.inf])
    if inf_mask.any().any():
        X = X.replace([np.inf, -np.inf], np.nan)
        X = X.fillna(X.mean())
    y = df2[target]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)
    
    # 1. Pipeline và param grid cho Random Forest
    rf_pipe = Pipeline([
        ("scaler", MinMaxScaler()), 
        ('model', RandomForestRegressor(criterion='absolute_error', random_state=7))
    ])
    
    rf_param_grid = {
        "model__n_estimators": [200, 300, 400],
        "model__max_depth": [20, 40, 60],
        "model__min_samples_leaf": [6, 8, 10],
        "model__min_samples_split": [6, 8, 10],
    }
    
    # 2. Pipeline và param grid cho XGBoost
    xgb_pipe = Pipeline([
        ("scaler", MinMaxScaler()), 
        ('model', XGBRegressor(objective='reg:absoluteerror', random_state=7))
    ])
    
    xgb_param_grid = {
        "model__n_estimators": [200, 250, 300],
        "model__max_depth": [12, 15, 18],
        "model__learning_rate": [0.1, 0.05],
        "model__subsample": [1.0, 0.8, 0.6],
        "model__colsample_bytree": [1.0],
    }
    
    models = [
        ('Random Forest', rf_pipe, rf_param_grid),
        ('XGBoost', xgb_pipe, xgb_param_grid)
    ]
    
    best_models = {}
    
    for name, pipe, param_grid in models:
        print(f'\n----- Optimizing {name} for {methods[i]} -----')
        search = GridSearchCV(pipe, param_grid, n_jobs=-1, cv=cv, 
                            scoring='neg_mean_absolute_error')
        search.fit(X_train, y_train)
        
        best_model = search.best_estimator_
        y_pred = best_model.predict(X_test)
        
        # Tính các metrics
        r2 = r2_score(y_test, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        mae = mean_absolute_error(y_test, y_pred)
        mape = mean_absolute_percentage_error(y_test, y_pred)
        
        print(f"\nBest parameters for {name}:")
        for param, value in search.best_params_.items():
            print(f"{param}: {value}")
        
        print("\nEvaluation Metrics on Test Set:")
        print(f"R2 Score: {r2:.4f}")
        print(f"RMSE: {rmse:.4f}")
        print(f"MAE: {mae:.4f}")
        print(f"MAPE: {mape:.4f}%")
        
        # Lấy feature importance
        feature_importance = get_feature_importance(best_model, features)
        if feature_importance is not None:
            print("\nTop 20 Important Features:")
            print(feature_importance.head(20).to_string(index=False))
        
        best_models[name] = {
            'model': best_model,
            'metrics': {
                'R2': r2,
                'RMSE': rmse,
                'MAE': mae,
                'MAPE': mape
            }
        }
    
    # So sánh kết quả tốt nhất giữa 2 mô hình
    print('\n----- Model Comparison -----')
    print(f"{'Model':<15} {'R2':>8} {'RMSE':>8} {'MAE':>8} {'MAPE':>8}")
    for name, result in best_models.items():
        metrics = result['metrics']
        print(f"{name:<15} {metrics['R2']:8.4f} {metrics['RMSE']:8.4f} " 
              f"{metrics['MAE']:8.4f} {metrics['MAPE']:8.4f}%")
    
    # Xác định best model dựa trên MAE
    best_model_name = min(best_models.items(), 
                         key=lambda x: x[1]['metrics']['MAE'])[0]
    print(f"\nBest model for {target} is {best_model_name} "
          f"(MAE = {best_models[best_model_name]['metrics']['MAE']:.4f})")
    
    # Lưu best model
    model_name = f"{methods[i]}_{best_model_name.replace(' ', '_')}"
    save_model(best_models[best_model_name]['model'], 'regression', model_name)


=== Building models for bps_pred ===

----- Optimizing Random Forest for BPS -----

Best parameters for Random Forest:
model__max_depth: 20
model__min_samples_leaf: 6
model__min_samples_split: 6
model__n_estimators: 200

Evaluation Metrics on Test Set:
R2 Score: 0.8809
RMSE: 12.3929
MAE: 9.1277
MAPE: 2.0122%

Top 20 Important Features:
  feature  importance
  #Accept    0.307170
   #Ether    0.095014
       #C    0.084697
     apol    0.080753
    SLogP    0.080645
 #Nitrile    0.063126
   #Donor    0.055659
    #AlHR    0.030920
     Vabc    0.028301
    #Bran    0.025401
   Radius    0.022323
     #C=O    0.018308
     #R=R    0.013486
       #O    0.011405
  #Ketone    0.010069
    #Ring    0.009186
      #SR    0.009150
     #SCR    0.007841
 #Epoxide    0.006145
#Aldehyde    0.005926

----- Optimizing XGBoost for BPS -----

Best parameters for XGBoost:
model__colsample_bytree: 1.0
model__learning_rate: 0.1
model__max_depth: 12
model__n_estimators: 200
model__subsample: 1.0

Evalu

TypeError: save_model() missing 1 required positional argument: 'method'

: 